In [1]:
import copy
import time

In [3]:
def generate_line_configurations(constraint, length):
    """
    根据给定的 Nonogram 行或列的数字提示 constraint（形如[3,2]）
    生成所有满足该提示的0/1可行配置列表。
    例如：constraint = [2], length = 5
    返回所有在5长度中恰好出现连续 2 个 1（黑格），其余为 0（白格）的配置
    """
    results = []
    
    def backtrack(current, idx, cidx):
        # current: 当前填充的0/1列表
        # idx: 正在填充的格子索引
        # cidx: constraint中的段落索引
        if cidx == len(constraint):
            # 所有段都已排布完, 剩余格子全置0
            if idx <= length:
                new_line = current + [0]*(length-len(current))
                results.append(new_line)
            return
        
        # 如果还没放第 cidx 段, 就需要在 idx 处开始放 constraint[cidx] 个1
        max_start = length - sum(constraint[cidx:]) - (len(constraint) - cidx - 1)  # 预留后面段的最少空格
        
        for start_pos in range(idx, max_start+1):
            # 先把 start_pos 到 start_pos+constraint[cidx]-1 置为 1
            # 前面补 (start_pos - len(current)) 个 0
            new_line = current + [0]*(start_pos-len(current)) + [1]*constraint[cidx]
            # 在该段后面加一个 0（如果还有下一段的话）
            if cidx < len(constraint) - 1:
                new_line.append(0)
            # 继续放下一个段
            backtrack(new_line, len(new_line), cidx+1)
    
    backtrack([], 0, 0)
    return results

In [5]:
def transpose(matrix):
    """ 转置二维列表 """
    return list(map(list, zip(*matrix)))

In [7]:
class NonogramSolverWithCP:
    def __init__(self, row_constraints, col_constraints):
        """
        row_constraints: 形如 [[3],[1,2],...] 每个元素是一行的数字提示
        col_constraints: 形如 [[2,1],[3],...] 每个元素是一列的数字提示
        """
        self.m = len(row_constraints)
        self.n = len(col_constraints)
        self.row_constraints = row_constraints
        self.col_constraints = col_constraints
        
        # 预生成所有行、列可能的填法
        self.row_poss = [
            generate_line_configurations(rc, self.n)
            for rc in self.row_constraints
        ]  # 每行对应的所有可行填法
        self.col_poss = [
            generate_line_configurations(cc, self.m)
            for cc in self.col_constraints
        ]  # 每列对应的所有可行填法
        
        # 最终解列表(可能多个解)
        self.solutions = []
    
    def propagate_constraints(self, row_poss, col_poss):
        """
        进行一轮约束传播，返回新的行可能集和列可能集、是否有更新。
        逻辑：某行的一种填法，和某列的一种填法不兼容，就要删除它们。
        - row_poss[i] 是当前第 i 行的所有可行填法，每个填法长度等于 self.n
        - col_poss[j] 是当前第 j 列的所有可行填法，每个填法长度等于 self.m
        """
        updated = False
        
        # 首先对行的每个可能填法，判断是否与列的可能填法兼容
        for i in range(self.m):
            new_row_configurations = []
            for row_cfg in row_poss[i]:
                compatible = True
                for j in range(self.n):
                    col_cfg_list = col_poss[j]
                    # row_cfg[j]是(行i)列j处的值，需要与所有可行的 col_cfg_list 中某些匹配
                    # 如果该值在 col_cfg_list 的所有配置的第 i 个位置都不匹配，则行cfg不兼容
                    if not any(col_cfg[i] == row_cfg[j] for col_cfg in col_cfg_list):
                        compatible = False
                        break
                if compatible:
                    new_row_configurations.append(row_cfg)
            if len(new_row_configurations) < len(row_poss[i]):
                updated = True
            row_poss[i] = new_row_configurations
        
        # 同理，对列的每个可能填法，判断是否与行的可能填法兼容
        for j in range(self.n):
            new_col_configurations = []
            for col_cfg in col_poss[j]:
                compatible = True
                for i in range(self.m):
                    row_cfg_list = row_poss[i]
                    # col_cfg[i]是(列j)行i处的值，需要与 row_cfg_list 的某些匹配
                    if not any(row_cfg[j] == col_cfg[i] for row_cfg in row_cfg_list):
                        compatible = False
                        break
                if compatible:
                    new_col_configurations.append(col_cfg)
            if len(new_col_configurations) < len(col_poss[j]):
                updated = True
            col_poss[j] = new_col_configurations
        
        return row_poss, col_poss, updated

    def is_consistent(self, row_poss, col_poss):
        """ 
        判断当前 row_poss 和 col_poss 是否还存在可行解 
        如果某行或列没有任何可行填法，则表示不一致 
        """
        for i in range(self.m):
            if not row_poss[i]:
                return False
        for j in range(self.n):
            if not col_poss[j]:
                return False
        return True

    def solve_logic(self, row_poss, col_poss):
        """
        不断进行约束传播，直到稳定或发现无解或已确定解的数目足够。
        如果无法一次性确定唯一解，就选一个最小可能集行/列进行回溯搜索。
        """
        # 反复进行约束传播，直到没有更新为止
        while True:
            row_poss, col_poss, updated = self.propagate_constraints(row_poss, col_poss)
            if not self.is_consistent(row_poss, col_poss):
                return  # 无解，直接返回
            if not updated:
                break  # 无更新，说明已到达稳定
        
        # 检查是否所有行都只剩1种可能，且所有列都只剩1种可能
        all_rows_single = all(len(x) == 1 for x in row_poss)
        all_cols_single = all(len(x) == 1 for x in col_poss)
        if all_rows_single and all_cols_single:
            # 这里理论上应该一致，拿 row_poss 就能拼出一个解
            solution_board = []
            for i in range(self.m):
                solution_board.append(row_poss[i][0])  # 取唯一的那一种
            # 验证一下与列的唯一填法相互匹配
            # 如果匹配则加入 solutions
            if self.verify_solution(solution_board):
                self.solutions.append(solution_board)
            return
        
        # 选择一个最少可能集的行或列进行回溯
        # 先在行中找可能性>1的最小者
        row_idx = None
        row_min = float('inf')
        for i in range(self.m):
            l = len(row_poss[i])
            if 1 < l < row_min:
                row_min = l
                row_idx = i
        
        # 同样在列中找
        col_idx = None
        col_min = float('inf')
        for j in range(self.n):
            l = len(col_poss[j])
            if 1 < l < col_min:
                col_min = l
                col_idx = j
        
        # 如果行的最小可能数更少，我们先对行进行回溯，否则对列进行回溯
        if row_idx is not None or col_idx is not None:
            if row_min <= col_min:
                # 对第 row_idx 行的填法进行分支
                for cfg in row_poss[row_idx]:
                    # 分支复制
                    new_row_poss = copy.deepcopy(row_poss)
                    new_col_poss = copy.deepcopy(col_poss)
                    # 将此行的填法固定为 [cfg]
                    new_row_poss[row_idx] = [cfg]
                    self.solve_logic(new_row_poss, new_col_poss)
            else:
                # 对第 col_idx 列进行分支
                for cfg in col_poss[col_idx]:
                    new_row_poss = copy.deepcopy(row_poss)
                    new_col_poss = copy.deepcopy(col_poss)
                    # 将此列的填法固定为 [cfg]
                    new_col_poss[col_idx] = [cfg]
                    self.solve_logic(new_row_poss, new_col_poss)

    def verify_solution(self, board):
        """
        最终验证 board 是否满足所有的行列约束。
        board 是一个二维数组 [row][col]，每个元素是 0 or 1。
        """
        # 检查行约束
        for i in range(self.m):
            if not self.check_line(board[i], self.row_constraints[i]):
                return False
        # 检查列约束
        transposed = transpose(board)
        for j in range(self.n):
            if not self.check_line(transposed[j], self.col_constraints[j]):
                return False
        return True
    
    def check_line(self, line, constraint):
        """
        与之前相同，用于验证某行或某列是否满足给定的数字提示。
        """
        blocks = []
        cur = 0
        for cell in line:
            if cell == 1:
                cur += 1
            else:
                if cur > 0:
                    blocks.append(cur)
                    cur = 0
        if cur > 0:
            blocks.append(cur)
        return blocks == constraint
    
    def solve(self):
        """ 对外的求解接口 """
        # 深拷贝一份 row_poss, col_poss，不修改原本的
        row_poss_cpy = copy.deepcopy(self.row_poss)
        col_poss_cpy = copy.deepcopy(self.col_poss)
        self.solve_logic(row_poss_cpy, col_poss_cpy)

        return self.solutions

In [9]:

# --------------------测试示例---------------------
if __name__ == "__main__":
    # 示例一：所有行列都是[5]，5x5全黑唯一解
    start_time = time.time()   
    row_constraints = [[1], [1], [1], [1], [1]] 
    col_constraints = [[1], [1], [1], [1], [1]] 
    solver = NonogramSolverWithCP(row_constraints, col_constraints)
    solver.solve()
    end_time = time.time()
    print(f"解的数量: {len(solver.solutions)},用时 {end_time - start_time:.8f} 秒，回溯+约束传播算法计算.")
    for idx, sol in enumerate(solver.solutions):
        print(f"解 #{idx+1}:")
        for row in sol:
            print(row)
        print()

解的数量: 120,用时 0.14160252 秒，回溯+约束传播算法计算.
解 #1:
[1, 0, 0, 0, 0]
[0, 1, 0, 0, 0]
[0, 0, 1, 0, 0]
[0, 0, 0, 1, 0]
[0, 0, 0, 0, 1]

解 #2:
[1, 0, 0, 0, 0]
[0, 1, 0, 0, 0]
[0, 0, 1, 0, 0]
[0, 0, 0, 0, 1]
[0, 0, 0, 1, 0]

解 #3:
[1, 0, 0, 0, 0]
[0, 1, 0, 0, 0]
[0, 0, 0, 1, 0]
[0, 0, 1, 0, 0]
[0, 0, 0, 0, 1]

解 #4:
[1, 0, 0, 0, 0]
[0, 1, 0, 0, 0]
[0, 0, 0, 1, 0]
[0, 0, 0, 0, 1]
[0, 0, 1, 0, 0]

解 #5:
[1, 0, 0, 0, 0]
[0, 1, 0, 0, 0]
[0, 0, 0, 0, 1]
[0, 0, 1, 0, 0]
[0, 0, 0, 1, 0]

解 #6:
[1, 0, 0, 0, 0]
[0, 1, 0, 0, 0]
[0, 0, 0, 0, 1]
[0, 0, 0, 1, 0]
[0, 0, 1, 0, 0]

解 #7:
[1, 0, 0, 0, 0]
[0, 0, 1, 0, 0]
[0, 1, 0, 0, 0]
[0, 0, 0, 1, 0]
[0, 0, 0, 0, 1]

解 #8:
[1, 0, 0, 0, 0]
[0, 0, 1, 0, 0]
[0, 1, 0, 0, 0]
[0, 0, 0, 0, 1]
[0, 0, 0, 1, 0]

解 #9:
[1, 0, 0, 0, 0]
[0, 0, 1, 0, 0]
[0, 0, 0, 1, 0]
[0, 1, 0, 0, 0]
[0, 0, 0, 0, 1]

解 #10:
[1, 0, 0, 0, 0]
[0, 0, 1, 0, 0]
[0, 0, 0, 1, 0]
[0, 0, 0, 0, 1]
[0, 1, 0, 0, 0]

解 #11:
[1, 0, 0, 0, 0]
[0, 0, 1, 0, 0]
[0, 0, 0, 0, 1]
[0, 1, 0, 0, 0]
[0, 0, 0, 1, 0]

解 

In [ ]:
for idx, sol in enumerate(solver.solutions):
    print(f"解 #{idx+1}:")
    for row in sol:
        line = "".join("■" if cell == 1 else " " for cell in row)
        print(line)
    print()